# Project: Ecommerce_Project EDA
## Day 2: Data Cleaning & Preprocessing

**Author:** Muhammed Riswan
**Date:** 2025-12-09

**Objectives:**
1. Fix data types (Dates & IDs).
2. Handle outliers (Review Scores).
3. Standardize categorical text.
4. Save the cleaned dataset for analysis.

In [100]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-commerce-dataset-for-practice/ecommerce_synthetic_dataset.csv


In [101]:
df = pd.read_csv("/kaggle/input/e-commerce-dataset-for-practice/ecommerce_synthetic_dataset.csv")

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   UserID              100000 non-null  int64  
 1   UserName            100000 non-null  object 
 2   Age                 100000 non-null  int64  
 3   Gender              100000 non-null  object 
 4   Country             100000 non-null  object 
 5   SignUpDate          100000 non-null  object 
 6   ProductID           100000 non-null  int64  
 7   ProductName         100000 non-null  object 
 8   Category            100000 non-null  object 
 9   Price               100000 non-null  float64
 10  PurchaseDate        100000 non-null  object 
 11  Quantity            100000 non-null  int64  
 12  TotalAmount         100000 non-null  float64
 13  HasDiscountApplied  100000 non-null  bool   
 14  DiscountRate        100000 non-null  float64
 15  ReviewScore         100000 non-null

In [103]:
df['UserID'] = df['UserID'].astype(str)
df['ProductID'] = df['ProductID'].astype(str)

In [104]:
df[['UserID', 'ProductID']].dtypes

UserID       object
ProductID    object
dtype: object

In [105]:
cols = ['SignUpDate','PurchaseDate','LastLogin']

for col in cols:
    df[col]  = pd.to_datetime(df[col],errors = 'coerce')
df[['SignUpDate','PurchaseDate','LastLogin']].dtypes

SignUpDate      datetime64[ns]
PurchaseDate    datetime64[ns]
LastLogin       datetime64[ns]
dtype: object

In [106]:
df.isna().sum()

UserID                0
UserName              0
Age                   0
Gender                0
Country               0
SignUpDate            0
ProductID             0
ProductName           0
Category              0
Price                 0
PurchaseDate          0
Quantity              0
TotalAmount           0
HasDiscountApplied    0
DiscountRate          0
ReviewScore           0
ReviewText            0
LastLogin             0
SessionDuration       0
DeviceType            0
ReferralSource        0
dtype: int64

### **Missing Value Analysis**
Check for null values across all columns to decide on imputation strategies.

**Observation:**
* The dataset contains **0 missing values** (`0 nulls`) across all 21 columns.

**Decision:**
> ✅ **No imputation or dropping was required** since the data is 100% complete.

In [107]:
print("Before Cleaning Score Range:", df['ReviewScore'].min(), "to", df['ReviewScore'].max())

Before Cleaning Score Range: -0.6 to 8.6


In [108]:
df['ReviewScore' ].value_counts()

ReviewScore
 4.0    4106
 4.1    4032
 3.9    4025
 4.2    3984
 3.8    3889
        ... 
-0.5       1
 8.1       1
 8.6       1
-0.6       1
-0.1       1
Name: count, Length: 85, dtype: int64

In [109]:
invalid_mask = (df['ReviewScore'] < 1) | (df['ReviewScore'] > 5)
df.loc[invalid_mask, 'ReviewScore'].value_counts()


ReviewScore
 5.1    2192
 5.2    1937
 5.3    1717
 5.4    1504
 5.5    1289
 5.6    1162
 5.7     926
 5.8     827
 5.9     647
 6.0     541
 6.1     460
 6.2     343
 6.3     291
 6.4     224
 6.5     177
 6.6     137
 6.7     107
 6.8      81
 6.9      57
 7.0      43
 0.9      35
 7.1      29
 7.2      28
 0.7      23
 0.8      22
 7.3      20
 0.6      14
 7.4      12
 0.5      10
 7.5       6
 7.6       6
 0.4       6
 7.7       5
 0.2       4
 8.0       3
 0.3       3
 7.9       2
 8.4       2
 8.6       1
-0.6       1
 8.1       1
-0.1       1
 7.8       1
-0.5       1
Name: count, dtype: int64

In [110]:
invalid_mask  =  (df['ReviewScore']<0)|(df['ReviewScore']>10)
df.loc[invalid_mask,'ReviewScore'].value_counts()


ReviewScore
-0.5    1
-0.6    1
-0.1    1
Name: count, dtype: int64

### **Observation: Review Score Scale**
Upon analyzing the distribution, the data shows a smooth pattern extending beyond 5.0 (e.g., values like 6.5, 7.8, 8.6).
* **Conclusion:** The `ReviewScore` is likely on a **0–10 scale**, not 1–5.
* **Action:** Values between 5.1 and 10.0 will be **kept** as valid.

### **Data Cleaning Action: Handling Negative Scores**
> **Decision:** Remove rows with negative scores.

**Reasoning:**
* **Keeping and converting to 0** introduces incorrect information → leads to wrong insights.
* The `ReviewScore` column contained **3 invalid negative values** (`-0.5`, `-0.6`, `-0.1`).
* Because these values are impossible in a 0–10 rating system and represent data entry errors, they were **removed** from the dataset.

In [111]:
df = df[(df['ReviewScore']>=0)&(df['ReviewScore']<=10)]
df['ReviewScore'].min()
df['ReviewScore'].max()

8.6

In [112]:

cols = ['Gender', 'Category', 'Country', 'DeviceType','ProductName','HasDiscountApplied','ReferralSource']
for col in cols:
    print(col, "→", df[col].unique())


Gender → ['Male' 'Female' 'Non-Binary']
Category → ['Books' 'Accessories' 'Apparel' 'Electronics']
Country → ['UK' 'Canada' 'Germany' 'USA' 'India' 'Australia']
DeviceType → ['Mobile' 'Tablet' 'Desktop']
ProductName → ['Shoes' 'T-shirt' 'Headphones' 'Smartphone' 'Laptop' 'Book' 'Watch']
HasDiscountApplied → [False  True]
ReferralSource → ['Social Media' 'Organic Search' 'Email Marketing' 'Ad Campaign']


### **Categorical Variable Inspection**
Before applying any cleaning, we reviewed the key categorical columns — `Gender`, `Category`, `Country`, `DeviceType`, `ProductName`, `HasDiscountApplied` and `ReferralSource` — to check for inconsistent text (e.g., casing issues, misspellings, or extra whitespace).

**Observation:**
After inspecting the unique values in each column, we confirmed that:
* All categories are properly formatted (Title Case).
* There are no duplicate category names (e.g., "Male" vs "male").
* No irregular text patterns were found.

**Result:**
> ✅ **No standardization was required** for categorical columns in this dataset.


In [113]:
df.info()
print(f"Final Shape: {df.shape}")

<class 'pandas.core.frame.DataFrame'>
Index: 99997 entries, 0 to 99999
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   UserID              99997 non-null  object        
 1   UserName            99997 non-null  object        
 2   Age                 99997 non-null  int64         
 3   Gender              99997 non-null  object        
 4   Country             99997 non-null  object        
 5   SignUpDate          99997 non-null  datetime64[ns]
 6   ProductID           99997 non-null  object        
 7   ProductName         99997 non-null  object        
 8   Category            99997 non-null  object        
 9   Price               99997 non-null  float64       
 10  PurchaseDate        99997 non-null  datetime64[ns]
 11  Quantity            99997 non-null  int64         
 12  TotalAmount         99997 non-null  float64       
 13  HasDiscountApplied  99997 non-null  bool          


In [114]:
duplicate_rows = df.duplicated().sum()
print("Duplicate Rows:", duplicate_rows)
print("Ensures that dataset doesn’t have repeated customers, purchases, or events.")

Duplicate Rows: 0
Ensures that dataset doesn’t have repeated customers, purchases, or events.


### **Outlier Detection: Why IQR over Z-Score?**

**1. Z-Score Limitations**
* Z-Score works best for **Normal Distributions** (bell curves).
* **Problem:** E-commerce data (Price, Quantity) is usually **skewed** (e.g., most items are cheap, a few are expensive). This distorts the Mean, making Z-Scores unreliable.

**2. Why We Use IQR (Interquartile Range)**


* **Robust:** It focuses on the stable **middle 50%** of data, ignoring extreme values.
* **Formula:** Outliers fall outside $Q1 - 1.5 \times IQR$ and $Q3 + 1.5 \times IQR$.
* **Best Fit:** Standard method for skewed retail and financial data.

In [115]:
numeric_cols = ['Price', 'Quantity', 'TotalAmount', 'SessionDuration', 'DiscountRate']

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower) | (df[col] > upper)]
    print(f"{col}: {len(outliers)} outliers")


Price: 0 outliers
Quantity: 0 outliers
TotalAmount: 723 outliers
SessionDuration: 0 outliers
DiscountRate: 0 outliers


### **Outlier Analysis (IQR Method)**

Checked for outliers in all numeric columns (Price, SessionDuration, etc.).

**Findings:**
* **TotalAmount:** 723 outliers detected.
* **All Other Columns:** 0 outliers found.

**Decision: Keep All Data**
We **kept** the 723 "outliers" in `TotalAmount` because they represent valid **high-value purchases** (VIP customers or bulk orders). Removing them would delete real revenue data.


In [117]:
df.to_csv('cleaned_day2.csv', index=False)

## ✅ Day 2 Summary — Cleaning & Preprocessing

- Date columns converted successfully.
- ID columns standardized to string type.
- ReviewScore corrected by removing 3 invalid negative values.
- Categorical variables inspected; no cleaning required.
- No duplicate rows found.
- IQR outlier detection done; 723 high-spend valid outliers retained.
- Dataset saved to `cleaned_day2.csv`.

Final Shape: 99,997 rows × 21 columns
